# COGS 108 - Data Checkpoint

# Names

- Asher Av
- Quoc-Zuy  Do
- Hector Gallo
- Jeremy Nurding
- Andres Villegas

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.


**Data Set Name: COVID-19 Data - US Counties from NYTimes**
- Link to Dataset: https://github.com/nytimes/covid-19-data
- Number of Obsevations: 2,170,941 
- <ins>Description of Dataset:</ins> This dataset is collated from  data across the U.S. by the New York Times and draws from the official reportings about the cumulative number of cases and deaths reported in each county and state across the U.S since the start of the COVID-19 pandemic. This dataset contains 6 columns of  data: date, county, state, fips, cases and  deaths. The FIPS column  crefers to a FIPS code, a geographic identifier that determines the location of the county the data was pulled from and makes it easy to associate with other datasets

**Data Set Name: Air Quality Across Countries in COVID-19**
- Link to Dataset: https://www.kaggle.com/aestheteaman01/air-quality-across-countries-in-covid19/version/3?select=USA.csv
- Number of Observations: 179,365
- <ins>Description of Dataset:</ins> This dataset shows the collected information of the air quality from 2020-2021 across countries including Brazil, Canada, France, Italy, India, and USA and their respective cities. The parameters that they use to show the air quality for each of the cities are the following: Carbon Monoxide (CO), Dew (dew), Humidity (humidity), Nitrogen Dioxide (NO2), Ozone (O3), Particulate Matter 10 (pm10, Particulate Matter 2.5 (pm25), Pressure (pressure), Sulphur Dioxide (SO2), Temperature, Wind Gusts, and Wind Speed.

**Data Set Name: Air Quality Statistics by County, 2020**
- Link to Dataset: https://www.epa.gov/air-trends/air-quality-cities-and-counties 
- Number of Observations: 1,144
- <ins>Description of Dataset:</ins> The data set “Air Quality Statistics by County, 2020” presents data of counties with highest levels of air pollutants across states in the United States.  This data set is organized alphabetically in rows by state, each county is assigned to its state accordingly and also arranged in alphabetical order.  There are a total of 1162 rows in this data set and out of the total of rows however 1144 rows contain observations of air pollutants for specific counties.  This dataset contains 13 columns from left to right as follows: State, County, County FIPS Code, 2010 population, CO, Pb, NO2 AM (ppb), NO2 hr (ppb), O3, PM10, PM2.5 Wtd AM, PM2.5 hr, and SO2.  These show the highest number of such pollutant in the given area.

# Setup

##### Import Modules

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# nytimes daily covid dataset
covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

# Kaggle air quality dataset
kagaq_df = pd.read_csv('https://raw.githubusercontent.com/asherbav/covid_pollution_files/main/USA.csv')

# EPA air quality dataset 
epa_df = pd.read_csv('https://raw.githubusercontent.com/asherbav/covid_pollution_files/main/epa_trends.csv')


# Data Cleaning

The first thing that we want to do is take a look at the original datasets to see what they look like. We first take a look at the COVID-19 Dataset entitled: "COVID-19 Data - US Counties from NYTimes". 

In [ ]:
covid_df

We then want to check if there are any null-values that we might want to get rid of  using isnull().sum. We see that there are a bunch of null values inside of the death's columns and the FIPS column. 
We also want to check the types of the columns to make sure that they are in the format that is desired for instance it seems that the dates were properly  converted to datetime objects, however the FIPS columns are floats instead of integers as expected.

In [ ]:
print(covid_df.isnull().sum())
print('-----------------') 
covid_df.dtypes


We however, will not be needing the deaths column for our analysis so we can drop the column entirely. We may want to keep the fips data however as creating a heatmaps of COVID-19 distribution across regions will allow us to analyze which regions are the most heavily impacted by the virus. This requires that we remove the entries where there are null values using dropna() from Pandas. 

We also want to change the type of the FIPS codes to integers so they are in the expected format.

In [ ]:
covid_df = covid_df.drop(['deaths'], axis = 1)
covid_df = covid_df.dropna()
covid_df['fips'] = covid_df['fips'].astype(int)


At this point we are done cleaning the COVID-19 dataset. We run isnull() again to double check that all the NULL/NaN values have been removed from out dataset properly. We also display the dataset and can see that the FIPS codes are now integers  as expected.  At this point the COVID-19 dataset has been cleaned and is now in a state where it can be used in our project.

In [ ]:
print(covid_df.isnull().sum())
print('-----------------') 
covid_df

We repeat the steps for the COVID dataset from above for the Kaggle dataset on Air Quality. We display the dataset to  get an idea of what it looks like, search for any NULL/NaN values, and then also check the types of all the objects to make sure if everything is in order.

In [ ]:
print(kagaq_df.isnull().sum())
print('-----------------') 
print(kagaq_df.dtypes)
print('-----------------') 
print(kagaq_df['Country'].value_counts())
kagaq_df

From the above analysis we can see that there are no null values, and that data types of each column is as expected for this dataset. This dataset has already clean and we don't have to do much from here. However, this dataset has an extraneous column and that is the "Country" column. This particular dataset only had dataset for the United States of America, therefore this column does not provide any additional useful information we can safely drop this column.

In [ ]:
kagaq_df = kagaq_df.drop(['Country'], axis = 1)

Next we redisplay the data set and confirm that the column was removed properly.

In [ ]:
kagaq_df